In [122]:
from scipy import io
import pprint
import numpy as np

In [123]:
class calculatePose:
    def __init__(self, viconDataPath: str, imuDataPath: str, imuParamPath: str) -> None:
        viconDataLoader = io.loadmat(viconDataPath)
        viconTS = viconDataLoader["ts"]
        viconTS = viconTS[0]
        # print(viconTS)
        viconData = viconDataLoader[
            "rots"
        ]  # Rotation Matrix with Z-Y-X Euler Angles Rotation
        # pprint.pprint(viconData)

        imuDataLoader = io.loadmat(imuDataPath)
        IMUTS = imuDataLoader["ts"]
        IMUTS = IMUTS[0]
        IMUData = imuDataLoader["vals"]  # [ax ay az wz wx wy]^T
        # pprint.pprint(IMUTS)
        # pprint.pprint(IMUData)

        imuParamLoader = io.loadmat(imuParamPath)
        imuParams = imuParamLoader["IMUParams"]

        self.performIMUAttitudeEstimation(IMUTS, IMUData, imuParams, viconTS, viconData)

    def calculateBias(self, IMUData):
        wxTotal = 0
        wyTotal = 0
        wzTotal = 0
        totalIterations = 300
        for i in range(totalIterations):
            imuData = [
                IMUData[3, i],
                IMUData[4, i],
                IMUData[5, i],
            ]
            wz, wx, wy = imuData
            wxTotal += wx
            wyTotal += wy
            wzTotal += wz
        bgx = wxTotal / totalIterations
        bgy = wyTotal / totalIterations
        bgz = wzTotal / totalIterations
        return bgx, bgy, bgz

    def convertRawValuesToSIUnits(
        self, IMUData, bax, bay, baz, sx, sy, sz, bgx, bgy, bgz
    ):
        adjustedIMUData = np.zeros(
            (len(IMUData[0]), 6), dtype=float
        )  # t,ax,zy,az, wx,wy,wz
        for i in range(len(IMUData[0])):
            imuData = [
                IMUData[0, i],
                IMUData[1, i],
                IMUData[2, i],
                IMUData[3, i],
                IMUData[4, i],
                IMUData[5, i],
            ]
            # pprint.pprint(imuData)
            ax, ay, az, wz, wx, wy = imuData
            tilde_ax = (ax + bax) / sx
            tilde_ay = (ay + bay) / sy
            tilde_az = (az + baz) / sz

            tilde_wx = (3300 / 1023) * (np.pi / 180) * 0.3 * (wx - bgx)
            tilde_wy = (3300 / 1023) * (np.pi / 180) * 0.3 * (wy - bgy)
            tilde_wz = (3300 / 1023) * (np.pi / 180) * 0.3 * (wz - bgz)

            adjustedIMUData[i] = (
                tilde_ax,
                tilde_ay,
                tilde_az,
                tilde_wx,
                tilde_wy,
                tilde_wz,
            )
        return adjustedIMUData

    def convertRotZYXToEulerAngles(self, viconData):
        eulerViconData = np.zeros((len(viconData), 3), dtype=float)
        for i, R in enumerate(viconData):
            sy = np.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
            singular = sy < 1e-6
            if not singular:
                x = np.arctan2(R[2, 1], R[2, 2])
                y = np.arctan2(-R[2, 0], sy)
                z = np.arctan2(R[1, 0], R[0, 0])
            else:
                x = np.arctan2(-R[1, 2], R[1, 1])
                y = np.arctan2(-R[2, 0], sy)
                z = 0
            eulerViconData[i] = x, y, z

    def alignData(self, imuTS, imuData, viconTS, viconData):
        useIMUTS = True

        if viconTS[0] > imuTS[0]:
            print("Vicon Initializes First. Using Vicon TS as base")
            useIMUTS = False
        else:
            print("IMU Initializes First. Using IMU TS as base")
        
        if(useIMUTS):
            for i in range(len(imuTS)-1):
                

    def performIMUAttitudeEstimation(
        self, imuTS, IMUData, imuParams, viconTS, viconData
    ):
        scaleParams, biasParams = imuParams
        sx, sy, sz = scaleParams
        bax, bay, baz = biasParams
        bgx, bgy, bgz = self.calculateBias(IMUData)

        metricIMUData = self.convertRawValuesToSIUnits(
            IMUData, bax, bay, baz, sx, sy, sz, bgx, bgy, bgz
        )
        eulerViconData = self.convertRotZYXToEulerAngles(viconData)
        alignedData = self.alignData(imuTS, metricIMUData, viconTS, eulerViconData)

In [124]:
set1 = calculatePose(
    "Phase1/Data/Train/Vicon/viconRot1.mat",
    "Phase1/Data/Train/IMU/imuRaw1.mat",
    "Phase1/IMUParams.mat"
)

[1.29663678e+09 1.29663678e+09 1.29663678e+09 ... 1.29663684e+09
 1.29663684e+09 1.29663684e+09]
array([1.29663678e+09, 1.29663678e+09, 1.29663678e+09, ...,
       1.29663684e+09, 1.29663684e+09, 1.29663684e+09])
IMU Initializes First. Using IMU TS as base
